<a href="https://colab.research.google.com/github/AnviTendulkar/Airline_Fare_Prediction/blob/master/fare_prediction5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler ,FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import re
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PowerTransformer ,OneHotEncoder
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_absolute_error

In [9]:
pd.set_option("display.max_columns" ,None)
pd.set_option("display.max_rows" ,None)

In [10]:
train_data = pd.read_excel(r'/content/drive/MyDrive/Project/Airline Fare  Prediction/Data_Train.xlsx')

In [11]:
train_data.head(3)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882


In [12]:
train_data[train_data['Duration']=='19h']

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
1634,Jet Airways,24/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 25 Jun,19h,2 stops,No info,13882
3089,Jet Airways,6/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 07 Jun,19h,2 stops,In-flight meal not included,11992
3926,Jet Airways,15/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 16 Jun,19h,2 stops,No info,13882
5863,Jet Airways,9/04/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Apr,19h,2 stops,No info,11546
6315,Jet Airways,12/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 13 Jun,19h,2 stops,No info,13882
8961,Jet Airways,6/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 07 Jun,19h,2 stops,No info,13882
9173,Jet Airways,27/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 28 Jun,19h,2 stops,No info,13882
10324,Jet Airways,21/03/2019,Banglore,New Delhi,BLR → BOM → DEL,05:45,00:45 19 Mar,19h,1 stop,In-flight meal not included,7832


In [13]:
train_data.dropna(inplace=True)

In [14]:
X = train_data.loc[:,train_data.columns !='Price']
y =train_data['Price']

In [15]:
x_train ,x_test ,y_train ,y_test = train_test_split(X,y ,test_size=0.20)

In [16]:
print(x_train.head(1))
print(pd.DataFrame(x_test.head(1)))

      Airline Date_of_Journey    Source Destination      Route Dep_Time  \
9172  Vistara       9/04/2019  Banglore       Delhi  BLR → DEL    07:00   

     Arrival_Time Duration Total_Stops Additional_Info  
9172        09:40   2h 40m    non-stop         No info  
                Airline Date_of_Journey Source Destination            Route  \
7921  Multiple carriers      21/03/2019  Delhi      Cochin  DEL → BOM → COK   

     Dep_Time Arrival_Time Duration Total_Stops Additional_Info  
7921    08:00        15:30   7h 30m      1 stop         No info  


In [17]:
def Arrival_Date_calculation(row):
    pattern = re.compile(r'(?:(\d+)h)?\s?(?:(\d+)m)?')
    match = pattern.match(row['Duration'])
    if match:
        hours = int(match.group(1)) if match.group(1) else 0
        minutes = int(match.group(2)) if match.group(2) else 0

    # Concatenate 'Date_of_Journey' and 'Dep_Time'
    date_of_journey = row['Date_of_Journey'] + ' ' + row['Dep_Time']

    # Convert the concatenated string to datetime
    journey_date_time = datetime.strptime(date_of_journey, "%d/%m/%Y %H:%M")

    # Calculate arrival date by adding 'Duration_hour' and 'Duration_minute' to 'Date_of_Journey'
    arrival_date = journey_date_time + timedelta(hours=hours, minutes=minutes)
    return arrival_date

def split_dep_time_in_hours_min(str1):
    lst = str1.split(':')
    return pd.Series({'Dep_Hour': lst[0], 'Dep_Minute': lst[1]})

In [18]:
x_train.head(1)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
9172,Vistara,9/04/2019,Banglore,Delhi,BLR → DEL,07:00,09:40,2h 40m,non-stop,No info


In [19]:
 df = x_train[['Airline','Date_of_Journey','Source','Destination','Dep_Time','Duration','Total_Stops']]

df.head(2)

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Duration,Total_Stops
9172,Vistara,9/04/2019,Banglore,Delhi,07:00,2h 40m,non-stop
7792,Air India,21/05/2019,Delhi,Cochin,18:45,24h 30m,2 stops


In [20]:
# score = clf.score(x_test, y_test)
# y_pred = clf.predict(x_test)
# mean_absolute_error(y_test,y_pred)
# clf.get_params(deep=True)

In [21]:
# clf[:-1].get_feature_names_out()

In [22]:

# from sklearn.impute import SimpleImputer
# from sklearn.compose import make_column_transformer
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.impute import SimpleImputer
# from sklearn.linear_model import LinearRegression


# SimpleImputer.get_feature_names_out = (lambda self, names=None:self.feature_names_in_)
# print(SimpleImputer.get_feature_names_out)
# num_pipeline = make_pipeline(SimpleImputer(), StandardScaler())
# transformer = make_column_transformer(
#     (num_pipeline, ["age", "height"]),
#     (OneHotEncoder(), ["city"]))
# pipeline = make_pipeline(transformer, LinearRegression())



# df = pd.DataFrame({"city": ["Rabat", "Tokyo", "Paris", "Auckland"],
#                    "age": [32, 65, 18, 24],
#                    "height": [172, 163, 169, 190],
#                    "weight": [65, 62, 54, 95]},
#                   index=["Alice", "Bunji", "Cécile", "Dave"])



# pipeline.fit(df, df["weight"])


# ## get pipeline feature names
# print(pipeline[:-1].get_feature_names_out())


# ## specify feature names as your columns
# pd.DataFrame(pipeline[:-1].transform(df),
#              columns=pipeline[:-1].get_feature_names_out(),
#              index=df.index)

In [23]:
# print(clf[:-1].get_feature_names_out())

In [24]:
###################
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class DateColumnExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        temp_df = pd.DataFrame()
        X['Arrival_Date'] = X.apply(Arrival_Date_calculation, axis=1)
        temp_df['Arrival_Year'] = X['Arrival_Date'].dt.year
        temp_df['Arrival_Month'] = X['Arrival_Date'].dt.month
        temp_df['Arrival_Day'] = X['Arrival_Date'].dt.day
        temp_df['Arrival_Hour'] = X['Arrival_Date'].dt.hour
        temp_df['Arrival_Minute'] = X['Arrival_Date'].dt.minute

        temp = X['Date_of_Journey'].copy()
        temp = pd.to_datetime(temp)
        # Extract year, month, and day into separate columns
        temp_df['Dep_Year'] = temp.dt.year
        temp_df['Dep_Month'] = temp.dt.month
        temp_df['Dep_Day'] = temp.dt.day

        temp_df[['Dep_Hour', 'Dep_Minute']] = X['Dep_Time'].apply(split_dep_time_in_hours_min)
        return temp_df

    def get_params(self, deep=True):
        return {}


In [25]:
#################################################

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
import pandas as pd

# Assume the DateColumnExtractor class is defined as provided in your previous message

# Your DataFrame
df = x_train[['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Dep_Time', 'Duration', 'Total_Stops']]

# Define the DateColumnExtractor
date_extractor = DateColumnExtractor()

# Define the ColumnTransformer
preprocessor = ColumnTransformer([
    ('one-hot', OneHotEncoder(drop='first', sparse_output=False, dtype='int16'), [0, 2, 3]),
    ('total_stop', FunctionTransformer(lambda x: x.replace({'1 stops': 1, '1 stop': 1, 'non-stop': 0, '2 stops': 2, '3 stops': 3, '4 stops': 4}),
                                      validate=False), [df.columns.get_loc('Total_Stops')]),
    ('date_extractor', date_extractor, ['Date_of_Journey', 'Dep_Time','Duration'])
], remainder='passthrough')

# Define the pipeline
pipe_clf = Pipeline([
    ('preprocessor', preprocessor)
])

# Fit and transform the data
transformed_data = pipe_clf.fit_transform(df)
print(pd.DataFrame(transformed_data).head(3))
print(transformed_data.shape)
# Concatenate the transformed data with the remaining columns
#transformed_df = pd.concat([pd.DataFrame(transformed_data), df.drop(['Date_of_Journey', 'Dep_Time', 'Total_Stops',], axis=1)], axis=1)

# Print the transformed DataFrame
#print(transformed_df.head(3))


<ipython-input-24-6564aef5afb5>:19: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  temp = pd.to_datetime(temp)


  0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 20    21 22  23  \
0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  1  0  0  0  0  2019  4   9   
1  1  0  0  0  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0  2  2019  5  22   
2  0  0  0  1  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0  1  2019  6  25   

   24  25    26 27  28  29  30  
0   9  40  2019  9   4  07  00  
1  19  15  2019  5  21  18  45  
2   4  25  2019  6  24  02  15  
(8545, 31)
   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20    21 22  23  \
0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  1  0  0  0  0  2019  4   9   
1  1  0  0  0  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0  2  2019  5  22   
2  0  0  0  1  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0  1  2019  6  25   

   24  25    26 27  28  29  30      Airline    Source Destination Duration  
0   9  40  2019  9   4  07  00       IndiGo  Banglore   New Delhi   2h 50m  
1  19  15  2019  5  21  18  45    Air India   Kolkata

In [26]:
# transformed_columns = preprocessor.transformers_[-1][-1] + preprocessor.get_feature_names_out()

# # Concatenate the transformed data with the remaining columns and set column names
# transformed_df = pd.concat([pd.DataFrame(transformed_data, columns=transformed_columns),
#                             df.drop(['Date_of_Journey', 'Dep_Time', 'Total_Stops'], axis=1)],
#                             axis=1)

# # Print the transformed DataFrame with column names
# print(transformed_df.head(3))

In [27]:
# ... (previous code)
#####################################
# Fit and transform the data
#transformed_data1 = pipe_clf.fit_transform(df)

# Get the names of the transformed columns
one_hot_columns = preprocessor.named_transformers_['one-hot'].get_feature_names_out(['Airline', 'Source', 'Destination'])
total_stop_columns = ['Total_Stops']  # You might need to update this based on your actual transformation

# Combine all transformed column names
transformed_columns = list(one_hot_columns) + total_stop_columns + ['Arrival_Year', 'Arrival_Month', 'Arrival_Day', 'Arrival_Hour', 'Arrival_Minute', 'Dep_Year', 'Dep_Month', 'Dep_Day', 'Dep_Hour', 'Dep_Minute']

# Concatenate the transformed data with the remaining columns and set column names
transformed_df = pd.concat([pd.DataFrame(transformed_data1, columns=transformed_columns),
                            df.drop(['Date_of_Journey', 'Dep_Time', 'Total_Stops'], axis=1)],
                            axis=1)

# Print the transformed DataFrame with column names
print(transformed_df.head(3))


<ipython-input-24-6564aef5afb5>:19: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  temp = pd.to_datetime(temp)


  Airline_Air India Airline_GoAir Airline_IndiGo Airline_Jet Airways  \
0                 0             0              0                   0   
1                 1             0              0                   0   
2                 0             0              0                   1   

  Airline_Jet Airways Business Airline_Multiple carriers  \
0                            0                         0   
1                            0                         0   
2                            0                         0   

  Airline_Multiple carriers Premium economy Airline_SpiceJet Airline_Trujet  \
0                                         0                0              0   
1                                         0                0              0   
2                                         0                0              0   

  Airline_Vistara Airline_Vistara Premium economy Source_Chennai Source_Delhi  \
0               1                               0              0        

In [28]:
df.head(3)

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Duration,Total_Stops
9172,Vistara,9/04/2019,Banglore,Delhi,07:00,2h 40m,non-stop
7792,Air India,21/05/2019,Delhi,Cochin,18:45,24h 30m,2 stops
2797,Jet Airways,24/06/2019,Delhi,Cochin,02:15,26h 10m,1 stop


In [29]:
df[df['Duration']=='19h']

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Duration,Total_Stops
5863,Jet Airways,9/04/2019,Delhi,Cochin,09:25,19h,2 stops
10324,Jet Airways,21/03/2019,Banglore,New Delhi,05:45,19h,1 stop
2,Jet Airways,9/06/2019,Delhi,Cochin,09:25,19h,2 stops
9173,Jet Airways,27/06/2019,Delhi,Cochin,09:25,19h,2 stops
6315,Jet Airways,12/06/2019,Delhi,Cochin,09:25,19h,2 stops
